# Ler arquivo CSV

In [38]:
import pandas as pd

In [39]:
filename = 'data/sales_data_simplified.csv'
df = pd.read_csv(filename, index_col=0)

df.head()

,Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Month,City
0,176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",4,Dallas (TX)
2,176559,Bose SoundSport Headphones,1,99.99,"682 Chestnut St, Boston, MA 02215",4,Boston (MA)
3,176560,Google Phone,1,600.00,"669 Spruce St, Los Angeles, CA 90001",4,Los Angeles (CA)
4,176560,Wired Headphones,1,11.99,"669 Spruce St, Los Angeles, CA 90001",4,Los Angeles (CA)
5,176561,Wired Headphones,1,11.99,"333 8th St, Los Angeles, CA 90001",4,Los Angeles (CA)


In [40]:
# mudar o indice
df = df.set_index('Order ID')
df.head()

,Product,Quantity Ordered,Price Each,Purchase Address,Month,City
Order ID,,,,,,
176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",4,Dallas (TX)
176559,Bose SoundSport Headphones,1,99.99,"682 Chestnut St, Boston, MA 02215",4,Boston (MA)
176560,Google Phone,1,600.00,"669 Spruce St, Los Angeles, CA 90001",4,Los Angeles (CA)
176560,Wired Headphones,1,11.99,"669 Spruce St, Los Angeles, CA 90001",4,Los Angeles (CA)
176561,Wired Headphones,1,11.99,"333 8th St, Los Angeles, CA 90001",4,Los Angeles (CA)


# Criar a tabela no banco

In [10]:
# importar biblioteca do banco
import sqlite3

In [44]:
# # conectando ao banco de dados em memória
# conn = sqlite3.connect(':memory:')

# conectando ao banco de dados
conn = sqlite3.connect('enterprise.db')

In [12]:
# salvar os dados no banco
df.to_sql(name='sales', con=conn)

/home/carlos/miniconda3/envs/synapse/lib/python3.7/site-packages/pandas/core/generic.py:2786: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [34]:
# testar se a tabela foi criada
!sqlite3 enterprise.db '.tables'

sales


In [35]:
!sqlite3 enterprise.db 'PRAGMA table_info(sales)'

0|index|INTEGER|0||0
1|Order ID|INTEGER|0||0
2|Product|TEXT|0||0
3|Quantity Ordered|INTEGER|0||0
4|Price Each|REAL|0||0
5|Purchase Address|TEXT|0||0
6|Month|INTEGER|0||0
7|City|TEXT|0||0
8|Sales|REAL|0||0


# Ler os dados do banco

In [50]:
df_read = pd.read_sql('SELECT * FROM sales', con=conn)
df_read.head()

,index,Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Month,City,Sales
0,0,176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",4,Dallas (TX),23.90
1,1,176559,Bose SoundSport Headphones,1,99.99,"682 Chestnut St, Boston, MA 02215",4,Boston (MA),99.99
2,2,176560,Google Phone,1,600.00,"669 Spruce St, Los Angeles, CA 90001",4,Los Angeles (CA),600.00
3,3,176560,Wired Headphones,1,11.99,"669 Spruce St, Los Angeles, CA 90001",4,Los Angeles (CA),11.99
4,4,176561,Wired Headphones,1,11.99,"333 8th St, Los Angeles, CA 90001",4,Los Angeles (CA),11.99


In [58]:
# selecionar parte das colunas
query = """
            SELECT "Order ID", "Product", "Quantity Ordered", "Price Each" 
            FROM sales;
        """

df_read = pd.read_sql(query, con=conn, index_col="Order ID")
df_read.head()

,Product,Quantity Ordered,Price Each
Order ID,,,
176558,USB-C Charging Cable,2,11.95
176559,Bose SoundSport Headphones,1,99.99
176560,Google Phone,1,600.00
176560,Wired Headphones,1,11.99
176561,Wired Headphones,1,11.99


In [27]:
# consulta com filtragem de dados
query = """
            SELECT *
            FROM sales
            WHERE "Product" = "USB-C Charging Cable";
        """

df_read = pd.read_sql(query, con=conn)
df_read

,Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Month,City
0,176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",4,Dallas (TX)
1,176562,USB-C Charging Cable,1,11.95,"381 Wilson St, San Francisco, CA 94016",4,San Francisco (CA)
2,176564,USB-C Charging Cable,1,11.95,"790 Ridge St, Atlanta, GA 30301",4,Atlanta (GA)
3,176573,USB-C Charging Cable,1,11.95,"214 Chestnut St, San Francisco, CA 94016",4,San Francisco (CA)
4,176574,USB-C Charging Cable,1,11.95,"20 Hill St, Los Angeles, CA 90001",4,Los Angeles (CA)
...,...,...,...,...,...,...,...
21898,259339,USB-C Charging Cable,2,11.95,"509 Park St, Austin, TX 73301",9,Austin (TX)
21899,259343,USB-C Charging Cable,1,11.95,"675 Maple St, Austin, TX 73301",9,Austin (TX)
21900,259350,USB-C Charging Cable,1,11.95,"519 Maple St, San Francisco, CA 94016",9,San Francisco (CA)
21901,259352,USB-C Charging Cable,1,11.95,"976 Forest St, San Francisco, CA 94016",9,San Francisco (CA)


In [59]:
# consulta criando uma nova coluna
query = """
            SELECT *, "Quantity Ordered" * "Price Each" AS "Sales"
            FROM sales;
        """

df_read = pd.read_sql(query, con=conn)
df_read

,index,Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Month,City,Sales,Sales
0,0,176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",4,Dallas (TX),23.90,23.90
1,1,176559,Bose SoundSport Headphones,1,99.99,"682 Chestnut St, Boston, MA 02215",4,Boston (MA),99.99,99.99
2,2,176560,Google Phone,1,600.00,"669 Spruce St, Los Angeles, CA 90001",4,Los Angeles (CA),600.00,600.00
3,3,176560,Wired Headphones,1,11.99,"669 Spruce St, Los Angeles, CA 90001",4,Los Angeles (CA),11.99,11.99
4,4,176561,Wired Headphones,1,11.99,"333 8th St, Los Angeles, CA 90001",4,Los Angeles (CA),11.99,11.99
...,...,...,...,...,...,...,...,...,...,...
185945,185945,259353,AAA Batteries (4-pack),3,2.99,"840 Highland St, Los Angeles, CA 90001",9,Los Angeles (CA),8.97,8.97
185946,185946,259354,iPhone,1,700.00,"216 Dogwood St, San Francisco, CA 94016",9,San Francisco (CA),700.00,700.00
185947,185947,259355,iPhone,1,700.00,"220 12th St, San Francisco, CA 94016",9,San Francisco (CA),700.00,700.00
185948,185948,259356,34in Ultrawide Monitor,1,379.99,"511 Forest St, San Francisco, CA 94016",9,San Francisco (CA),379.99,379.99


# Reescrevendo os dados no banco

In [ ]:
# salvar os dados no banco
df_read.to_sql(name='sales', con=conn) #error

In [31]:
# salvar os dados no banco
df_read.to_sql(name='sales', con=conn, if_exists='replace')

/home/carlos/miniconda3/envs/synapse/lib/python3.7/site-packages/pandas/core/generic.py:2786: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [32]:
df_read = pd.read_sql('SELECT * FROM sales', con=conn)
df_read.head()

,index,Order ID,Product,Quantity Ordered,Price Each,Purchase Address,Month,City,Sales
0,0,176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",4,Dallas (TX),23.90
1,1,176559,Bose SoundSport Headphones,1,99.99,"682 Chestnut St, Boston, MA 02215",4,Boston (MA),99.99
2,2,176560,Google Phone,1,600.00,"669 Spruce St, Los Angeles, CA 90001",4,Los Angeles (CA),600.00
3,3,176560,Wired Headphones,1,11.99,"669 Spruce St, Los Angeles, CA 90001",4,Los Angeles (CA),11.99
4,4,176561,Wired Headphones,1,11.99,"333 8th St, Los Angeles, CA 90001",4,Los Angeles (CA),11.99


In [33]:
# fechar a conexão com o banco
conn.close()